### Route Rescheduler ###

In [11]:
import sys
from pathlib import Path

src_path = Path("../src").resolve()  # adjust relative to the notebook location
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

In [12]:
# Load required libraries and preprocessed data
import pandas as pd
from simulator import simulate_new_jobs
from evaluation import evaluate_feasible_insertions

# Load preprocessed CSVs
df_trips = pd.read_csv("../data/processed/trips.csv", parse_dates=['od_start_time', 'od_end_time'])
df_routes = pd.read_csv("../data/processed/routes.csv", parse_dates=['route_start_time', 'route_end_time'])

# Simulate 5 new jobs (disruptions)
new_jobs = simulate_new_jobs(df_trips, n_jobs=5)

# Evaluate where these jobs can be reassigned
candidates = evaluate_feasible_insertions(new_jobs, df_routes)

# Display results
print("Simulated Jobs:")
display(new_jobs)

print("Feasible Reassignments:")
display(candidates.head(10))

Simulated Jobs:


,route_schedule_uuid,trip_uuid,segment_actual_time,segment_osrm_time,segment_osrm_distance,actual_time,osrm_time,osrm_distance,cutoff_factor,start_scan_to_end_scan,od_start_time,od_end_time,source_center,destination_center,trip_duration_minutes,job_id
0,thanos::sroute:731fb539-6925-459c-8074-17df8b6...,trip-153701473441299363,522.0,265.0,320.8230,1855.0,952.0,1156.1401,75.166667,5196.0,2018-09-15 12:32:14.413233,2018-09-16 10:00:44.468604,IND110037AAM,IND110037AAM,1288.500923,NEW_JOB_0
1,thanos::sroute:cec0e49a-6807-4202-ac88-9cb360d...,trip-153856463279004477,166.0,34.0,38.1535,382.0,76.0,89.0427,16.666667,594.0,2018-10-03 11:03:52.790326,2018-10-03 14:22:36.502023,IND400102AAC,IND421302AAG,198.728528,NEW_JOB_1
2,thanos::sroute:31171483-ca90-40a9-94d5-39309e6...,trip-153756713929871939,67.0,56.0,55.5595,113.0,96.0,95.1523,11.666667,316.0,2018-09-22 00:01:22.843581,2018-09-22 02:40:49.510654,IND695582AAA,IND695614AAA,159.444451,NEW_JOB_2
3,thanos::sroute:16dbdb8b-1ca6-4d26-a45c-18d378e...,trip-153774227248337690,308.0,167.0,195.7269,706.0,386.0,458.0759,40.500000,1330.0,2018-09-23 22:37:52.483617,2018-09-24 08:28:26.744761,IND627005AAA,IND627005AAA,590.571019,NEW_JOB_3
4,thanos::sroute:9b079f7c-3b70-4619-81f7-14ee2a2...,trip-153850290910260618,936.0,38.0,40.3387,1230.0,112.0,111.8140,21.750000,4340.0,2018-10-02 17:55:09.102819,2018-10-03 12:00:24.676698,IND000000ACB,IND110037AAK,1085.259565,NEW_JOB_4


Feasible Reassignments:


,job_id,route_schedule_uuid,new_total_time,new_total_distance,original_time,original_distance,added_time,added_distance
0,NEW_JOB_1,thanos::sroute:02447388-0297-4e63-9a71-3589741...,271.431773,50.4424,72.703245,12.2889,198.728528,38.1535
1,NEW_JOB_1,thanos::sroute:036f372d-28d8-4d19-877c-6277077...,286.175111,48.3572,87.446583,10.2037,198.728528,38.1535
2,NEW_JOB_1,thanos::sroute:053e2fea-873d-4842-8e3a-edd5690...,239.741026,50.1204,41.012498,11.9669,198.728528,38.1535
3,NEW_JOB_1,thanos::sroute:06b37046-b91b-4f4d-9aac-e799aa0...,500.350478,261.1842,301.621950,223.0307,198.728528,38.1535
4,NEW_JOB_1,thanos::sroute:0a05c445-d943-4f05-82ed-fd90a6d...,275.893831,49.9321,77.165303,11.7786,198.728528,38.1535
5,NEW_JOB_1,thanos::sroute:0b175e79-8058-4ea1-8be8-29569ca...,370.892308,62.4528,172.163780,24.2993,198.728528,38.1535
6,NEW_JOB_1,thanos::sroute:11448700-b0d6-4009-afa6-79d2b56...,372.354908,51.3882,173.626380,13.2347,198.728528,38.1535
7,NEW_JOB_1,thanos::sroute:12d71f58-4272-434c-bb34-f8f9111...,366.538485,71.4716,167.809957,33.3181,198.728528,38.1535
8,NEW_JOB_1,thanos::sroute:1692165b-034a-4c4a-9e7e-2e88121...,429.095615,111.3421,230.367087,73.1886,198.728528,38.1535
9,NEW_JOB_1,thanos::sroute:19f6e955-0102-4569-9e02-db60b18...,239.857227,53.8421,41.128699,15.6886,198.728528,38.1535
